In [9]:
import requests
import pandas as pd
from datetime import datetime
import os
import time

# ================================
# CONFIGURAÇÕES
# ================================
API_KEY = "BF90A113-835D-11F0-BDE5-4201AC1DC121"

# Região geográfica (bounding box)
# Exemplo: litoral de Santa Catarina
NW_LAT, NW_LNG = 5.27, -74.02    # canto Noroeste da Amazônia
SE_LAT, SE_LNG = -33.75, -34.79  # canto Sudeste perto do oceano Atlântico


# Período de interesse
START_DATE = "2025-08-01"
END_DATE   = "2025-08-01"

# Média temporal em minutos (ex: 60 = média horária)
AVERAGE = 60  

# Pasta para salvar os arquivos
OUTPUT_DIR = "purpleair_region_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ================================
# FUNÇÃO PARA LISTAR SENSORES NA REGIÃO
# ================================
def get_sensors_in_region(api_key, nw_lat, nw_lng, se_lat, se_lng):
    """Busca todos os sensores dentro de um bounding box"""
    url = "https://api.purpleair.com/v1/sensors"
    headers = {"X-API-Key": api_key}
    
    params = {
        "fields": "sensor_index,name,latitude,longitude,pm2.5_atm,pm1.0_atm,pm10.0_atm,last_seen",
        "location_type": 0,
        "nwlng": nw_lng,
        "nwlat": nw_lat,
        "selng": se_lng,
        "selat": se_lat,
    }
    
    r = requests.get(url, headers=headers, params=params)
    
    if r.status_code == 200:
        data = r.json()
        fields = data["fields"]
        values = data["data"]
        df = pd.DataFrame(values, columns=fields)
        df["last_seen"] = pd.to_datetime(df["last_seen"], unit="s")
        return df
    else:
        print(f"❌ Erro ao buscar sensores: {r.status_code} {r.text}")
        return pd.DataFrame()

# ================================
# FUNÇÃO PARA BAIXAR HISTÓRICO DE UM SENSOR
# ================================
def download_history(sensor_id, start_date, end_date, average=60, api_key=None):
    url = f"https://api.purpleair.com/v1/sensors/{sensor_id}/history"
    
    params = {
        "fields": "pm2.5_atm,pm1.0_atm,pm10.0_atm,temperature,humidity,pressure",
        "start_timestamp": int(datetime.fromisoformat(start_date).timestamp()),
        "end_timestamp": int(datetime.fromisoformat(end_date).timestamp()),
        "average": average,
    }
    
    headers = {"X-API-Key": api_key}
    r = requests.get(url, headers=headers, params=params)
    
    if r.status_code == 200:
        data = r.json()
        if "data" in data and data["data"]:
            fields = data["fields"]
            values = data["data"]
            df = pd.DataFrame(values, columns=fields)
            df["time_stamp"] = pd.to_datetime(df["time_stamp"], unit="s")
            return df
        else:
            print(f"⚠️ Sem dados para o sensor {sensor_id}")
            return None
    else:
        print(f"❌ Erro {r.status_code} no sensor {sensor_id}: {r.text}")
        return None

# ================================
# EXECUÇÃO
# ================================
print("📍 Buscando sensores na região...")
sensors_df = get_sensors_in_region(API_KEY, NW_LAT, NW_LNG, SE_LAT, SE_LNG)

if sensors_df.empty:
    print("Nenhum sensor encontrado.")
else:
    print(f"✅ {len(sensors_df)} sensores encontrados.")

    for _, row in sensors_df.iterrows():
        sid = int(row["sensor_index"])
        name = row["name"].replace(" ", "_") if row["name"] else f"sensor_{sid}"
        
        print(f"\n📥 Baixando histórico do sensor {sid} ({name})...")
        df_hist = download_history(sid, START_DATE, END_DATE, AVERAGE, API_KEY)
        
        if df_hist is not None:
            filename = os.path.join(OUTPUT_DIR, f"{name}_{sid}.csv")
            df_hist.to_csv(filename, index=False)
            print(f"   ✅ Dados salvos em {filename}")
        
        time.sleep(1)  # evitar excesso de requisições


📍 Buscando sensores na região...
✅ 303 sensores encontrados.

📥 Baixando histórico do sensor 262813 (PELD-TANG)...
⚠️ Sem dados para o sensor 262813

📥 Baixando histórico do sensor 275450 (Vielas_da_Água_Preta)...
⚠️ Sem dados para o sensor 275450

📥 Baixando histórico do sensor 287170 (GEOHealth_TAC_GregorioA)...
⚠️ Sem dados para o sensor 287170

📥 Baixando histórico do sensor 287176 (GEOHealth_MAD_PuertoM_Wasai)...
⚠️ Sem dados para o sensor 287176

📥 Baixando histórico do sensor 25541 (MPAC_PTA_01_Sec.infraestrutura)...
⚠️ Sem dados para o sensor 25541

📥 Baixando histórico do sensor 25551 (MPAC_FIJ_01_promotoria)...
⚠️ Sem dados para o sensor 25551

📥 Baixando histórico do sensor 25891 (MPAC_BJR_01_promotoria)...
⚠️ Sem dados para o sensor 25891

📥 Baixando histórico do sensor 27841 (MPAC_BRL_01_promotoria)...
⚠️ Sem dados para o sensor 27841

📥 Baixando histórico do sensor 31091 (MPAC_TRC_02_ifac)...
⚠️ Sem dados para o sensor 31091

📥 Baixando histórico do sensor 31103 (MPAC_SNM